Create smaller batches, make strtified splits


In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 71kB 3.3MB/s 
     |████████████████████████████████| 1.3MB 8.5MB/s 
     |████████████████████████████████| 890kB 41.6MB/s 
     |████████████████████████████████| 1.1MB 45.3MB/s 
     |████████████████████████████████| 2.9MB 45.6MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.9-cp36-none-any.whl size=101036 sha256=980e09f520ecbf0eac3c57dddb3a862c29c81705841f4b6a66eb0b9de00c6abd
  Stored in directory: /root/.cache/pip/wheels/fc/89/43/f2f5bc00b03ef9724b0f6254a97eaf159a4c4ddc024b33e07a
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=3d059bd647ee4872084c6181bff07ed56a9635b4ec7b5d09cff5e565ab6707f0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [ ]:
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from tqdm import tqdm_notebook

from tensorflow.keras import layers, Sequential
from tensorflow import keras
import tensorflow as tf

import pandas as pd
import numpy as np
import os

pd.options.display.max_colwidth = 100

In [ ]:
os.chdir("/content/drive/MyDrive/SKBKonturTest")

In [ ]:
test = pd.read_csv('Data/test_data.tsv', sep='\t', index_col=0)
train = pd.read_csv('Data/train_data.tsv', sep='\t', index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
train_data, valid_data = train_test_split(train, test_size=0.2, stratify=train['answer'], random_state=42, shuffle=True)

In [ ]:
model_trans_name = 'LaBSE'
model_trans = SentenceTransformer(model_trans_name)

In [ ]:
def get_embeddings_and_save(data, embed_model, dir_name, batch_size):
  process_simultanious_n = 500_000

  n_bathes_per_iter = (process_simultanious_n // batch_size)
  process_simultanious_n = n_bathes_per_iter * batch_size
  n_iters = (np.ceil(len(data) / float(process_simultanious_n))).astype(np.int)

  for n_iter in tqdm_notebook(range(n_iters)):
    data_process = data[n_iter * process_simultanious_n : (n_iter + 1) * process_simultanious_n]

    eng_embedings = embed_model.encode(data_process['eng_name'].to_list(), show_progress_bar=False,
                                       batch_size=256, device='cuda:0', num_workers=4)
    ru_embedings = embed_model.encode(data_process['ru_name'].to_list(), show_progress_bar=False,
                                      batch_size=256, device='cuda:0', num_workers=4)
    
    X_process = np.concatenate((ru_embedings, eng_embedings), axis=1)
    y_process = data_process['answer'].astype(np.bool)

    n_batches = (np.ceil(len(data_process) / float(batch_size))).astype(np.int)

    for batch in range(n_batches):
      X_batch = X_process[batch * batch_size : (batch + 1) * batch_size]
      y_batch = y_process[batch * batch_size : (batch + 1) * batch_size]

      np.save(f'embedded_data/{dir_name}/X_batch_{n_iter * n_bathes_per_iter + batch}.npy', X_batch)
      np.save(f'embedded_data/{dir_name}/y_batch_{n_iter * n_bathes_per_iter + batch}.npy', y_batch)

In [ ]:
batch_size = 1024

get_embeddings_and_save(train_data, model_trans, 'train', batch_size)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
get_embeddings_and_save(valid_data, model_trans, 'val', batch_size)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


## Modeling

In [ ]:
class My_Custom_Generator(keras.utils.Sequence) :
  def __init__(self, batch_size, dir_name) :
    self.batch_size = batch_size
    self.dir_name = dir_name
    

  def __len__(self) :
    return 2 # len(os.listdir(f"embedded_data/{self.dir_name}")) // 2
  
  
  def __getitem__(self, batch) :
    batch_x = np.load(f'embedded_data/{self.dir_name}/X_batch_{batch}.npy')
    batch_y = np.load(f'embedded_data/{self.dir_name}/y_batch_{batch}.npy')
    return batch_x, batch_y

In [ ]:
batch_size = 1024

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(128, input_shape=(1536,) , activation="relu", name="Layer1"),
        layers.Dense(32, activation="relu", name="Layer2"),
        layers.Dense(1, activation='sigmoid' ,name="SigmoidLayer"),
    ]
)

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy(), keras.metrics.Precision(), keras.metrics.Recall()],
)

In [ ]:
training_batch_generator = My_Custom_Generator(batch_size, 'train')
validation_batch_generator = My_Custom_Generator(batch_size, 'val')

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, 
                                               mode="min", restore_best_weights=True)
checkpoint = keras.callbacks.ModelCheckpoint('trained_models/model_best.h5', verbose=1, 
                                             monitor='val_loss', save_best_only=True, mode='min')  

In [ ]:
history = model.fit(x=training_batch_generator,
                    epochs=30,
                    verbose=1,
                    validation_data=validation_batch_generator,
                    callbacks=[early_stopping, checkpoint]
                    )

Epoch 1/10
3112/3112 [==============================] - 5366s 2s/step - loss: 0.0610 - binary_accuracy: 0.9835 - precision: 0.9448 - recall: 0.6808 - val_loss: 0.0309 - val_binary_accuracy: 0.9920 - val_precision: 0.9549 - val_recall: 0.8660
Epoch 2/10
3112/3112 [==============================] - 5237s 2s/step - loss: 0.0283 - binary_accuracy: 0.9926 - precision: 0.9593 - recall: 0.8750 - val_loss: 0.0285 - val_binary_accuracy: 0.9926 - val_precision: 0.9397 - val_recall: 0.8953
Epoch 3/10
3112/3112 [==============================] - 5282s 2s/step - loss: 0.0255 - binary_accuracy: 0.9933 - precision: 0.9635 - recall: 0.8866 - val_loss: 0.0271 - val_binary_accuracy: 0.9929 - val_precision: 0.9728 - val_recall: 0.8697
Epoch 4/10
 610/3112 [====>.........................] - ETA: 54:26 - loss: 0.0236 - binary_accuracy: 0.9938 - precision: 0.9680 - recall: 0.8957

In [ ]:
model.fit(x=validation_batch_generator,
          epochs=10,
          verbose=1,
          )

Epoch 1/10
778/778 [==============================] - 309s 397ms/step - loss: 0.1159 - binary_accuracy: 0.9656 - precision: 0.9122 - recall: 0.2770
Epoch 2/10
778/778 [==============================] - 14s 18ms/step - loss: 0.0439 - binary_accuracy: 0.9887 - precision: 0.9341 - recall: 0.8104
Epoch 3/10
778/778 [==============================] - 13s 17ms/step - loss: 0.0339 - binary_accuracy: 0.9914 - precision: 0.9531 - recall: 0.8546
Epoch 4/10
778/778 [==============================] - 13s 17ms/step - loss: 0.0298 - binary_accuracy: 0.9924 - precision: 0.9601 - recall: 0.8702
Epoch 5/10
778/778 [==============================] - 13s 17ms/step - loss: 0.0274 - binary_accuracy: 0.9929 - precision: 0.9636 - recall: 0.8797
Epoch 6/10
778/778 [==============================] - 13s 17ms/step - loss: 0.0256 - binary_accuracy: 0.9934 - precision: 0.9655 - recall: 0.8873
Epoch 7/10
778/778 [==============================] - 13s 17ms/step - loss: 0.0240 - binary_accuracy: 0.9938 - precision: 

In [ ]:
history = model.fit(x=validation_batch_generator,
                    epochs=10,
                    verbose=1,
                    validation_data=validation_batch_generator,
                    callbacks=[early_stopping, checkpoint]
                    )

Epoch 1/10
1/2 [==============>...............] - ETA: 0s - loss: 0.1554 - binary_accuracy: 0.9639 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00
Epoch 00001: val_loss improved from inf to 0.14892, saving model to trained_models/model_best.h5
2/2 [==============================] - 0s 153ms/step - loss: 0.1491 - binary_accuracy: 0.9658 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: 0.1489 - val_binary_accuracy: 0.9658 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/10
1/2 [==============>...............] - ETA: 0s - loss: 0.1427 - binary_accuracy: 0.9678 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00
Epoch 00002: val_loss improved from 0.14892 to 0.14872, saving model to trained_models/model_best.h5
2/2 [==============================] - 0s 56ms/step - loss: 0.1489 - binary_accuracy: 0.9658 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: 0.1487 - val_binary_accuracy: 0.9658 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/1

In [ ]:
history.history

{'binary_accuracy': [0.96533203125,
  0.9658203125,
  0.9658203125,
  0.9658203125,
  0.9658203125,
  0.9658203125,
  0.9658203125,
  0.9658203125,
  0.9658203125,
  0.9658203125],
 'loss': [0.6183249354362488,
  0.5445506572723389,
  0.4686732888221741,
  0.3950635492801666,
  0.329170823097229,
  0.2730850577354431,
  0.22949877381324768,
  0.1969877928495407,
  0.17555108666419983,
  0.1627693772315979],
 'precision_1': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'recall_1': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'val_binary_accuracy': [0.9658203125,
  0.9658203125,
  0.9658203125,
  0.9658203125,
  0.9658203125,
  0.9658203125,
  0.9658203125,
  0.9658203125,
  0.9658203125,
  0.9658203125],
 'val_loss': [0.5635513067245483,
  0.48754847049713135,
  0.41279375553131104,
  0.3443722426891327,
  0.28574615716934204,
  0.23878489434719086,
  0.20379427075386047,
  0.17992278933525085,
  0.16525551676750183,
  0.15751402080059052],
 'val_precision_1': [0.0, 0.0, 

## Check on valid data

In [ ]:
ru_embedings = model_trans.encode(valid_data['ru_name'].to_list(), show_progress_bar=True,
                                  batch_size=256, device='cuda:0', num_workers=4)

eng_embedings = model_trans.encode(valid_data['eng_name'].to_list(), show_progress_bar=True,
                                  batch_size=256, device='cuda:0', num_workers=4)

X_val = np.concatenate((ru_embedings, eng_embedings), axis=1)
y_val = valid_data['answer'].astype(np.int8)

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f9fe05f65c0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1174, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f9fe05f65c0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dat

In [ ]:
def get_model(input_shape):
  model = keras.Sequential(
      [
          layers.Dense(128, input_shape=input_shape , activation="relu", name="Layer1"),
          layers.Dense(32, activation="relu", name="Layer2"),
          layers.Dense(1, activation='sigmoid' ,name="SigmoidLayer"),
      ]
  )

  model.compile(
      optimizer=keras.optimizers.Adam(),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=[keras.metrics.BinaryAccuracy(), keras.metrics.Precision(), keras.metrics.Recall()],
  )
  return model

In [ ]:
model = get_model(input_shape=(1536,))
model.fit(X_val, y_val, batch_size=512, epochs=5)

Epoch 1/5
1556/1556 [==============================] - 9s 6ms/step - loss: 0.1171 - binary_accuracy: 0.9671 - precision_3: 0.8671 - recall_3: 0.3344
Epoch 2/5
1556/1556 [==============================] - 10s 6ms/step - loss: 0.0423 - binary_accuracy: 0.9882 - precision_3: 0.9289 - recall_3: 0.8048
Epoch 3/5
1556/1556 [==============================] - 9s 6ms/step - loss: 0.0342 - binary_accuracy: 0.9907 - precision_3: 0.9464 - recall_3: 0.8441
Epoch 4/5
1556/1556 [==============================] - 9s 6ms/step - loss: 0.0303 - binary_accuracy: 0.9918 - precision_3: 0.9557 - recall_3: 0.8613
Epoch 5/5
1556/1556 [==============================] - 9s 6ms/step - loss: 0.0282 - binary_accuracy: 0.9924 - precision_3: 0.9589 - recall_3: 0.8722


In [ ]:
model = get_model(input_shape=(1536,))
model.fit(X_val, y_val, batch_size=1024, epochs=5)

Epoch 1/5
778/778 [==============================] - 6s 7ms/step - loss: 0.1377 - binary_accuracy: 0.9607 - precision_2: 0.9455 - recall_2: 0.1532
Epoch 2/5
778/778 [==============================] - 5s 7ms/step - loss: 0.0458 - binary_accuracy: 0.9882 - precision_2: 0.9338 - recall_2: 0.7993
Epoch 3/5
778/778 [==============================] - 5s 7ms/step - loss: 0.0353 - binary_accuracy: 0.9912 - precision_2: 0.9505 - recall_2: 0.8525
Epoch 4/5
778/778 [==============================] - 5s 7ms/step - loss: 0.0310 - binary_accuracy: 0.9924 - precision_2: 0.9584 - recall_2: 0.8720
Epoch 5/5
778/778 [==============================] - 5s 7ms/step - loss: 0.0284 - binary_accuracy: 0.9931 - precision_2: 0.9619 - recall_2: 0.8844


In [ ]:
model = get_model(input_shape=(1536,))
model.fit(X_val, y_val, batch_size=2048, epochs=5)

Epoch 1/5
389/389 [==============================] - 3s 9ms/step - loss: 0.1807 - binary_accuracy: 0.9532 - precision_4: 0.0476 - recall_4: 0.0011
Epoch 2/5
389/389 [==============================] - 3s 9ms/step - loss: 0.0741 - binary_accuracy: 0.9743 - precision_4: 0.9312 - recall_4: 0.4761
Epoch 3/5
389/389 [==============================] - 3s 9ms/step - loss: 0.0431 - binary_accuracy: 0.9890 - precision_4: 0.9386 - recall_4: 0.8127
Epoch 4/5
389/389 [==============================] - 3s 9ms/step - loss: 0.0345 - binary_accuracy: 0.9913 - precision_4: 0.9545 - recall_4: 0.8511
Epoch 5/5
389/389 [==============================] - 3s 9ms/step - loss: 0.0300 - binary_accuracy: 0.9926 - precision_4: 0.9634 - recall_4: 0.8710


In [ ]:
model = get_model(input_shape=(1536,))
model.fit(X_val, y_val, batch_size=4096, epochs=5)

Epoch 1/5
195/195 [==============================] - 2s 12ms/step - loss: 0.2037 - binary_accuracy: 0.9513 - precision_7: 0.0391 - recall_7: 0.0026
Epoch 2/5
195/195 [==============================] - 2s 13ms/step - loss: 0.1633 - binary_accuracy: 0.9541 - precision_7: 0.8451 - recall_7: 0.0016
Epoch 3/5
195/195 [==============================] - 2s 12ms/step - loss: 0.0937 - binary_accuracy: 0.9674 - precision_7: 0.9222 - recall_7: 0.3166
Epoch 4/5
195/195 [==============================] - 2s 12ms/step - loss: 0.0591 - binary_accuracy: 0.9839 - precision_7: 0.9190 - recall_7: 0.7114
Epoch 5/5
195/195 [==============================] - 2s 13ms/step - loss: 0.0482 - binary_accuracy: 0.9873 - precision_7: 0.9300 - recall_7: 0.7815


In [ ]:
model = get_model(input_shape=(1536,))
model.fit(X_val, y_val, batch_size=10_000, epochs=5)

Epoch 1/5
80/80 [==============================] - 2s 24ms/step - loss: 0.2356 - binary_accuracy: 0.9465 - precision_5: 0.0414 - recall_5: 0.0075
Epoch 2/5
80/80 [==============================] - 2s 23ms/step - loss: 0.1873 - binary_accuracy: 0.9541 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00
Epoch 3/5
80/80 [==============================] - 2s 24ms/step - loss: 0.1861 - binary_accuracy: 0.9541 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00
Epoch 4/5
80/80 [==============================] - 2s 23ms/step - loss: 0.1822 - binary_accuracy: 0.9541 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00
Epoch 5/5
80/80 [==============================] - 2s 23ms/step - loss: 0.1714 - binary_accuracy: 0.9541 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00


In [ ]:
model = get_model(input_shape=(1536,))
model.fit(X_val, y_val, batch_size=50_000, epochs=5)

Epoch 1/5
16/16 [==============================] - 2s 99ms/step - loss: 0.3958 - binary_accuracy: 0.9084 - precision_6: 0.0445 - recall_6: 0.0486
Epoch 2/5
16/16 [==============================] - 1s 92ms/step - loss: 0.1980 - binary_accuracy: 0.9541 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00
Epoch 3/5
16/16 [==============================] - 2s 98ms/step - loss: 0.1930 - binary_accuracy: 0.9541 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00
Epoch 4/5
16/16 [==============================] - 2s 94ms/step - loss: 0.1890 - binary_accuracy: 0.9541 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00
Epoch 5/5
16/16 [==============================] - 1s 90ms/step - loss: 0.1881 - binary_accuracy: 0.9541 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00
